This is the code for the Cosine Similarity recommendation system. The limitations of this is:
1. Only take book data into account, and doesn't compare user-to-user
2. The amount of memory and computing power is large, with the whole book dataset python needs to allocate ~80Gb. Currently to get around this I'm using 20000 instances of book data.

*Tunable parameters:
- Number of training instances
- m: maximum number of books to consider for recommendations (take m books with highest rating from user and consider them for similarity)


In [13]:
import pandas as pd
import numpy as np

import nltk
from nltk.corpus import stopwords
from brs_data_preprocessing import get_preprocessed_data as preproc, merged_book_ratings as merge

nltk.download('stopwords')


[nltk_data] Downloading package stopwords to
[nltk_data]     /home/legoeuro/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [14]:
usersDf, bookDf, ratingDf = preproc('input/Users.csv', 'input/Books.csv', 'input/Ratings.csv')

#Data concatenation
bookInfo = []
for _, row in bookDf.iterrows():
    bookInfo.append(f"{row['Book-Title']} {row['Book-Author']} {row['Publisher']}")
bookDf["info"] = bookInfo

#configs
bookDf = bookDf.iloc[:20000]
ratingDf['ISBN'] = ratingDf['ISBN'].astype(str)
bookDf['ISBN'] = bookDf["ISBN"].astype(str)
bookDf['id'] = bookDf.index


Add index for later steps 

Preprocess and clean book information
.

In [15]:

#from https://www.kaggle.com/code/muhammadayman/recommendation-system-using-cosine-similarity#Feature-Engineering
stop = stopwords.words('english')
def preprocess(column):
    #make all words with lower letters
    column = column.str.lower()
    #getting rid of any punctution
    column = column.str.replace('http\S+|www.\S+|@|%|:|,|', '', case=False)
    #spliting each sentence to words to apply previous funtions on them 
    word_tokens = column.str.split()
    keywords = word_tokens.apply(lambda x: [item for item in x if item not in stop])
    #assemble words of each sentence again and assign them in new column
    for i in range(len(keywords)):
        keywords[i] = " ".join(keywords[i])
        column = keywords

    return column
bookDf['info'] = preprocess(bookDf['info'])
# bookDf['Book-Title'] = preprocess(bookDf['Book-Title'])
bookDf.head(5)

,ISBN,Book-Title,Book-Author,Year-Of-Publication,Publisher,info,id
0,0195153448,Classical Mythology,Mark P. O. Morford,2002,Oxford University Press,classical mythology mark p. o. morford oxford ...,0
1,0002005018,Clara Callan,Richard Bruce Wright,2001,HarperFlamingo Canada,clara callan richard bruce wright harperflamin...,1
2,0060973129,Decision in Normandy,Carlo D'Este,1991,HarperPerennial,decision normandy carlo d'este harperperennial,2
3,0374157065,Flu: The Story of the Great Influenza Pandemic...,Gina Bari Kolata,1999,Farrar Straus Giroux,flu: story great influenza pandemic 1918 searc...,3
4,0393045218,The Mummies of Urumchi,E. J. W. Barber,1999,W. W. Norton &amp; Company,mummies urumchi e. j. w. barber w. w. norton &...,4


In [16]:
# from sklearn import preprocessing

# mms = preprocessing.MinMaxScaler()

# bookDf['Year-Of-Publication'] = (bookDf['Year-Of-Publication'] - bookDf['Year-Of-Publication'].mean()) / bookDf['Year-Of-Publication'].std() 

Apply cosine vectorizer to all books; Find user top 10 books that they have not read

In [17]:
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics.pairwise import cosine_similarity
from scipy.spatial.distance import pdist

CV = CountVectorizer()
titleVect = CV.fit_transform(bookDf['info'])
# authVect = CV.fit_transform(bookDf['Book-Author'])
# pubVect = CV.fit_transform(bookDf['Publisher'])

titleSim = cosine_similarity(titleVect)

normalize = 10
bookRatingMerge = merge(ratings_df=ratingDf, books_df=bookDf)
def recommendFromUser(userId, n, m):
    """
    userId: id of the user in the dataset
    n: number of recommendations
    m: maximum number of books to consider for recommendations (take m books with highest rating from user and consider them for similarity)
       made with the assumption that book-rating only goes from 1 to 10
    """
    userRatings = bookRatingMerge[bookRatingMerge['User-ID'] == userId]
    userRatings = userRatings[userRatings['Book-Rating'] > 5]

    ratingTruncated = userRatings.nlargest(m, 'Book-Rating')
    print(ratingTruncated.to_markdown())
    recommendations = []
    for _, row in bookDf.iterrows():
        bookId = row['ISBN']
        bookIndex = bookDf[bookDf['ISBN'] == bookId].index[0]
        sim = 0
        isBookRead = False
        for _, ratingRow in ratingTruncated.iterrows():
            ratingBookId = ratingRow['ISBN']
            ratingBookIndex = bookDf[bookDf['ISBN'] == ratingBookId].index[0]
            if bookIndex == ratingBookIndex:
                isBookRead = True
                break
            #weight by rating
            sim += ratingRow['Book-Rating']/(normalize) * titleSim[bookIndex][ratingBookIndex]
        if (not isBookRead):
            recommendations.append((bookId, sim)) 
    recommendations.sort(key=lambda x: x[1], reverse=True)
    return recommendations[:n]

#Example: get the top recommendation for the user with id 276709
recommendation = recommendFromUser(276709, 10, 10)
mappedRec = map(lambda x: (bookDf[bookDf['ISBN'] == x[0]]['Book-Title'].values[0], x[1]), recommendation)
print(list(mappedRec))



|        |       ISBN | Book-Title                                       | Book-Author       |   Year-Of-Publication | Publisher   | info                                                                      |    id |   User-ID |   Book-Rating |
|-------:|-----------:|:-------------------------------------------------|:------------------|----------------------:|:------------|:--------------------------------------------------------------------------|------:|----------:|--------------:|
| 344029 | 0515107662 | The Sherbrooke Bride (Bride Trilogy (Paperback)) | Catherine Coulter |                  1996 | Jove Books  | sherbrooke bride (bride trilogy (paperback)) catherine coulter jove books | 15978 |    276709 |            10 |
[('The Scottish Bride (Bride Trilogy (Paperback))', 0.9090909090909093), ('The Heiress Bride (Bride Trilogy (Paperback))', 0.9090909090909093), ('The Sherbrooke Twins', 0.6154574548966638), ('The Cove', 0.5393598899705937), ('The Maze', 0.5393598899705937), ('The E